# 1. Extraction des données (lxml)

Ce notebook extrait les paragraphes des comptes rendus de l’Assemblée nationale à partir des fichiers XML, en utilisant la bibliothèque lxml (qui a ici donné des résultats plus simples que la précédente version).


Sans doute moins robuste, moins de gestion des erreurs et exceptions, mais plus simple, lisible, et sans doute OK sur les données assemblée qui sont stables / propres.

In [ ]:
# TODO: s'assurer que le passage vers lxml est ok

In [ ]:
import os
import glob
from lxml import etree
import pandas as pd


def extraire_paragraphes_lxml(fichier_xml: str) -> pd.DataFrame:
    """
    Extrait les paragraphes d'un fichier XML de compte rendu en utilisant lxml.
    """
    try:
        tree = etree.parse(fichier_xml)
        root = tree.getroot()
        ns = {'ns': 'http://schemas.assemblee-nationale.fr/referentiel'}

        meta = {
            'UID': root.findtext('ns:uid', namespaces=ns),
            'SeanceRef': root.findtext('ns:seanceRef', namespaces=ns),
            'SessionRef': root.findtext('ns:sessionRef', namespaces=ns)
        }
        meta_tags = ['dateSeance', 'dateSeanceJour', 'numSeanceJour', 'numSeance', 'typeAssemblee',
                     'legislature', 'session', 'nomFichierJo']
        for tag in meta_tags:
            meta[tag] = root.findtext(f'.//ns:{tag}', namespaces=ns)

        meta['President'] = root.findtext('.//ns:presidentSeance', namespaces=ns)

        rows = []

        for para in root.xpath('.//ns:paragraphe', namespaces=ns):
            # Naviguer vers le <point> parent
            point = para.getparent()
            while point is not None and point.tag != f"{{{ns['ns']}}}point":
                point = point.getparent()

            point_id = point.get('id_syceron') if point is not None else None
            point_type = point.get('code_grammaire') if point is not None else None
            point_title = point.findtext('ns:texte', namespaces=ns) if point is not None else None
            valeur_odj = point.get('valeur_ptsodj') if point is not None else None

            texte_elem = para.find('ns:texte', namespaces=ns)
            texte = ''.join(texte_elem.itertext()).strip() if texte_elem is not None else None
            stime = texte_elem.get('stime') if texte_elem is not None else None

            orateur = para.find('.//ns:orateur', namespaces=ns)
            nom = orateur.findtext('ns:nom', namespaces=ns) if orateur is not None else None
            qualite = orateur.findtext('ns:qualite', namespaces=ns) if orateur is not None else None
            id_orateur = orateur.findtext('ns:id', namespaces=ns) if orateur is not None else None

            rows.append({
                'UID': meta['UID'],
                'SeanceRef': meta['SeanceRef'],
                'SessionRef': meta['SessionRef'],
                'DateSeance': meta['dateSeance'],
                'DateSeanceJour': meta['dateSeanceJour'],
                'NumSeanceJour': meta['numSeanceJour'],
                'NumSeance': meta['numSeance'],
                'TypeAssemblee': meta['typeAssemblee'],
                'Legislature': meta['legislature'],
                'Session': meta['session'],
                'NomFichierJO': meta['nomFichierJo'],
                'President': meta['President'],
                'Titre_general': point_title,
                # 'Sous_titre': '',  # not in this version, get back to original if needed
                # 'Contexte_hierarchique': '',  # not in this version, get back to original if needed
                # 'Section_courante': '',  # not in this version, get back to original if needed
                # 'Sujet_point': '', # not in this version, get back to original if needed
                'Valeur_ODJ': valeur_odj,
                'Point_ID': point_id,
                'Point_type': point_type,
                'ID_paragraphe': para.get('id_syceron'),
                'Ordre_seance': para.get('ordre_absolu_seance'),
                'Code_grammaire': para.get('code_grammaire'),
                'Code_style': para.get('code_style'),
                'Code_parole': para.get('code_parole'),
                'Role_debat': para.get('roledebat'),
                'Nom_orateur': nom,
                'Qualite_orateur': qualite,
                'ID_orateur': id_orateur,
                'stime': stime,
                'Texte': texte
            })

        return pd.DataFrame(rows)

    except Exception as e:
        print(f"❌ Erreur dans {fichier_xml} : {e}")
        return pd.DataFrame()


def traiter_dossier_compte_rendu_lxml(dossier_path: str, pattern: str = "*.xml") -> pd.DataFrame:
    """
    Traite tous les fichiers XML d'un dossier avec la fonction extraire_paragraphes_lxml().
    """
    fichiers = glob.glob(os.path.join(dossier_path, pattern))
    if not fichiers:
        print(f"❌ Aucun fichier XML trouvé dans {dossier_path}")
        return pd.DataFrame()

    all_dfs = []
    total = len(fichiers)
    print(f"🔄 Traitement de {total} fichiers XML...\n")

    for i, fichier in enumerate(fichiers, 1):
        nom = os.path.basename(fichier)
        print(f"[{i}/{total}] 📄 {nom}...", end=" ")

        df = extraire_paragraphes_lxml(fichier)
        if not df.empty:
            print(f"✅ {len(df)} lignes")
            all_dfs.append(df)
        else:
            print("⚠️ Vide ou erreur")

    if all_dfs:
        df_final = pd.concat(all_dfs, ignore_index=True)
        print(f"\n✅ Export terminé : {len(df_final)} lignes consolidées")
        return df_final
    else:
        return pd.DataFrame()


# # Exemple d'utilisation :
# if __name__ == "__main__":
#     dossier = "../data/raw/16-xml/compteRendu/"
#     df = traiter_dossier_compte_rendu_lxml(dossier)
#     df.to_csv("assemblee_extrait_lxml.csv", index=False, encoding='utf-8')
#     print(f"\n✅ Export CSV : ({df.shape[0]} lignes)"))


In [6]:
df = traiter_dossier_compte_rendu_lxml("../data/raw/16-xml/compteRendu/")
df.to_csv("../data/interim/extract_16.csv", index=False, encoding='utf-8')
print(f"\n✅ Export CSV : ({df.shape[0]} lignes)")

🔄 Traitement de 605 fichiers XML...

[1/605] 📄 CRSANR5L16S2023O1N173.xml... ✅ 633 lignes
[2/605] 📄 CRSANR5L16S2023O1N167.xml... ✅ 467 lignes
[3/605] 📄 CRSANR5L16S2023O1N198.xml... ✅ 402 lignes
[4/605] 📄 CRSANR5L16S2023E1N010.xml... ✅ 818 lignes
[5/605] 📄 CRSANR5L16S2024O1N117.xml... ✅ 678 lignes
[6/605] 📄 CRSANR5L16S2024O1N103.xml... ✅ 857 lignes
[7/605] 📄 CRSANR5L16S2023E1N004.xml... ✅ 646 lignes
[8/605] 📄 CRSANR5L16S2023O1N239.xml... ✅ 452 lignes
[9/605] 📄 CRSANR5L16S2023O1N205.xml... ✅ 118 lignes
[10/605] 📄 CRSANR5L16S2023O1N211.xml... ✅ 296 lignes
[11/605] 📄 CRSANR5L16S2024O1N088.xml... ✅ 403 lignes
[12/605] 📄 CRSANR5L16S2023O1N007.xml... ✅ 535 lignes
[13/605] 📄 CRSANR5L16S2023O1N013.xml... ✅ 808 lignes
[14/605] 📄 CRSANR5L16S2024O1N063.xml... ✅ 106 lignes
[15/605] 📄 CRSANR5L16S2024O1N077.xml... ✅ 760 lignes
[16/605] 📄 CRSANR5L16S2024O1N076.xml... ✅ 834 lignes
[17/605] 📄 CRSANR5L16S2024O1N062.xml... ✅ 322 lignes
[18/605] 📄 CRSANR5L16S2023O1N012.xml... ✅ 688 lignes
[19/605] 📄 CRSANR5